In [1]:
import argparse

import numpy as np
import tensorflow as tf
import keras.backend as K

import matplotlib.pyplot as plt

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from keras.regularizers import l2

import glob
import os
import cv2
import keras


import time
import argparse

from tqdm import tqdm

from keras.models import load_model, Model
from importlib import reload  


In [2]:
pwd

'C:\\Users\\fjdur\\Documents\\upc-july\\notebooks'

In [4]:
dataset = "cifar"

In [5]:
model_direction = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
if(dataset == 'gtsrb'):
    model_direction = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06"
elif(dataset == 'intel'):
    model_direction = "C:/Users/fjdur/Documents/upc-july/models/intel_model_21_10"
elif(dataset == 'mnist'):
    model_direction = "C:/Users/fjdur/Documents/MNIST project/models2"
elif(dataset == 'cifar'):
    model_direction = "C:/Users/fjdur/Documents/upc-july/models/model_02"


In [19]:
cd C:/Users/fjdur/Documents/utils/

C:\Users\fjdur\Documents\utils


In [20]:
# utils for project
import utils_guided_retraining as utils

In [21]:
pwd

'C:\\Users\\fjdur\\Documents\\utils'

In [22]:
from importlib import reload  
utils = reload(utils)

2.5.0
keras
2.5.0


In [23]:
cd ../MNIST project/

C:\Users\fjdur\Documents\MNIST project


In [24]:
x_train,y_train = utils.get_data(dataset,"Train")
x_val,y_val = utils.get_data(dataset,"Val")
x_test,y_test = utils.get_data(dataset,"Test")

In [25]:
x_train_and_adversary,y_train_and_adversary = utils.get_data(dataset,"Train_and_adversary")

In [26]:
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)
print("---")
print(x_train[0].shape)
print(x_train_and_adversary.shape)
print(y_train_and_adversary.shape)

(41999, 32, 32, 1)
(41999, 10)
(12000, 32, 32, 1)
(12000, 10)
(6001, 32, 32, 1)
(6001, 10)
---
(32, 32, 1)
(47999, 32, 32, 1)
(47999, 10)


In [27]:
num_classes = y_train_and_adversary.shape[-1]
print(num_classes)

10


In [28]:
#here editing


In [29]:
x_adversary_training = x_train_and_adversary[len(x_train):]
print(len(x_adversary_training))
y_adversary_training = y_train_and_adversary[len(y_train):]

print(len(y_adversary_training))

6000
6000


In [30]:
# Obtaining adversarial examples for testing 
x_test_and_adversary,y_test_and_adversary = utils.get_adversarial_data(dataset,"Test_adversarial") 

In [31]:
print(x_test_and_adversary.shape)
print(y_test_and_adversary.shape)

(12001, 32, 32, 1)
(12001, 10)


In [32]:
x_adversary_test_fgsm = x_test_and_adversary[len(x_test):]
print(len(x_adversary_test_fgsm))
y_adversary_test_fgsm = y_test_and_adversary[len(y_test):]

print(len(y_adversary_test_fgsm))

6000
6000


In [15]:
#x_test_and_adversary_jsma,y_test_and_adversary_jsma = utils.get_adversarial_data('Test_jsma') 

## ----

In [16]:
# Original model 
#model_dir = "C:/Users/fjdur/Desktop/upc/project_notebooks/github_project/DL_notebooks/models/tf_model_25-06"

In [17]:
#model_original = utils.My_model(True, model_dir)

In [33]:
# Loading model already trained
#model_dir = "C:/Users/fjdur/Documents/upc-july/models/tf_model_25-06/"
model = keras.models.load_model(model_direction)

In [34]:
model.evaluate(x_test,y_test)

188/188 [==============================] - 37s 31ms/step - loss: 0.5240 - accuracy: 0.8259


[0.524002730846405, 0.8258623480796814]

In [35]:
model.evaluate(x_test_and_adversary,y_test_and_adversary)

376/376 [==============================] - 12s 31ms/step - loss: 2.3668 - accuracy: 0.5769


[2.3667662143707275, 0.5768685936927795]

## Obtaining new LSA and DSA values 
- Required module 'deepimportance_code_release'

In [36]:
cd "D:deepimportance_code_release"

D:\deepimportance_code_release


In [37]:
# from 07_notebook
from utils import filter_val_set, get_trainable_layers
from utils import generate_adversarial, filter_correct_classifications
from coverages.idc import ImportanceDrivenCoverage
from coverages.neuron_cov import NeuronCoverage
from coverages.tkn import DeepGaugeLayerLevelCoverage
from coverages.kmn import DeepGaugePercentCoverage
from coverages.ss import SSCover
from coverages.sa import SurpriseAdequacy


In [38]:
trainable_layers = get_trainable_layers(model)
non_trainable_layers = list(set(range(len(model.layers))) - set(trainable_layers))
print('Trainable layers: ' + str(trainable_layers))
print('Non trainable layers: ' + str(non_trainable_layers))

Trainable layers: [0, 1, 3, 4, 7, 12, 14, 15]
Non trainable layers: [2, 5, 6, 8, 9, 10, 11, 13]


In [39]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_79 (Conv2D)           (None, 32, 32, 16)        160       
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_85 (Dropout)         (None, 8, 8, 64)        

In [42]:
subject_layer = trainable_layers[-2]
layer_names = [model.layers[-2].name]
upper_bound = 2000
layer_names

['dense_46']

In [43]:
subject_layer

14

In [44]:
from coverages import sa
from datetime import datetime

In [45]:
pwd

'D:\\deepimportance_code_release'

In [46]:
print("check ",dataset)

check  cifar


In [47]:
start_time = datetime.now()
#surprise of all inputs wrt to the original inputs
target_lsa = sa.fetch_lsa(model, x_train, x_train_and_adversary,
                                   "conf_03_aug", layer_names,
                                   num_classes, True,
                                   1e-05, './sa_'+dataset, "model_03_aug")


end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

[train] Model serving
16
329/329 [==============================] - 32s 94ms/step
[train] Processing ATs
Layer: dense_46
train ATs is saved at ./sa_cifar\model_03_aug_train_dense_46_ats.npy
[conf_03_aug] Model serving
16
375/375 [==============================] - 36s 95ms/step
[conf_03_aug] Processing ATs
Layer: dense_46
conf_03_aug ATs is saved at ./sa_cifar\model_03_aug_conf_03_aug_dense_46_ats.npy
yes
dict_keys([9, 6, 4, 2, 8, 1, 5, 0, 7, 3])
The number of removed columns: 6
[conf_03_aug] Fetching LSA
Duration: 0:08:49.566418


In [48]:
pwd

'D:\\deepimportance_code_release'

In [49]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
np.save(save_dir,np.array(target_lsa))

In [50]:
# editing here

In [51]:
start_time = datetime.now()

target_dsa = sa.fetch_dsa(model, x_train, x_train_and_adversary,
                                   "conf_02_aug", layer_names,
                                   num_classes, True,
                                   './sa_'+dataset,  "model_02_aug")
end_time = datetime.now()
print('Duration: {}'.format(end_time - start_time))

[train] Model serving
16
329/329 [==============================] - 32s 95ms/step
[train] Processing ATs
Layer: dense_46
train ATs is saved at ./sa_cifar\model_02_aug_train_dense_46_ats.npy
[conf_02_aug] Model serving
16
375/375 [==============================] - 37s 95ms/step
[conf_02_aug] Processing ATs
Layer: dense_46
conf_02_aug ATs is saved at ./sa_cifar\model_02_aug_conf_02_aug_dense_46_ats.npy
[conf_02_aug] Fetching DSA
Duration: 0:48:25.458844


In [60]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/dsa_values.npy"
np.save(save_dir,np.array(target_dsa))

In [42]:
#loading
loading =False
if (loading == True):
    save_dir = "C:/Users/fjdurlop/Documents/upc-july/data/"+dataset+"/lsa_values.npy"
    target_lsa = np.load(save_dir)

    save_dir = "C:/Users/fjdurlop/Documents/upc-july/data/"+dataset+"/dsa_values.npy"
    target_dsa = np.load(save_dir)

In [43]:
#print(target_dsa[48980:49010])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.4198453e-07, 0.103650354, 0.110504985, 0.11916372, 0.09978015, 0.15995812, 0.10416289, 0.13666278, 0.08570866, 0.18017817, 0.12767863, 0.11186311]


In [44]:
#print(target_dsa[49180:49210])

[0.16274497, 0.101583555, 0.12083733, 0.14051229, 0.11820864, 0.112263255, 0.085314415, 0.098417014, 0.13010219, 0.11316547, 0.11753961, 0.10477573, 0.101970434, 0.14521681, 0.110462874, 0.14807995, 0.12886639, 0.12255348, 0.11675078, 0.10050738, 0.114456624, 0.14356047, 0.14551164, 0.15918754, 0.0727881, 0.13202144, 0.11643102, 0.20601466, 0.11816457, 0.13781092]


In [45]:
#print(target_dsa[40180:40210])

[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


## Random

In [53]:
import random
random_indexes =list(range(len(x_train_and_adversary)))
random.shuffle(random_indexes)
print(random_indexes[:10])
print(len(random_indexes))

[7474, 10003, 6626, 17602, 21101, 4905, 14248, 13128, 43125, 31673]
47999


In [54]:

save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/random_values.npy"

np.save(save_dir,np.array(random_indexes))

In [61]:
cd D:/backup_UPC/upc/project_notebooks/TestSelection

D:\backup_UPC\upc\project_notebooks\TestSelection


In [62]:
cd ..

D:\backup_UPC\upc\project_notebooks


In [57]:
pwd

'D:\\backup_UPC\\upc\\project_notebooks'

In [63]:
import TestSelection
from TestSelection.utils.ncoverage import NCoverage # https://github.com/TestSelection/TestSelection

In [64]:
model.summary()

Model: "sequential_13"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_79 (Conv2D)           (None, 32, 32, 16)        160       
_________________________________________________________________
conv2d_80 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
max_pooling2d_73 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_81 (Conv2D)           (None, 16, 16, 64)        18496     
_________________________________________________________________
conv2d_82 (Conv2D)           (None, 16, 16, 64)        36928     
_________________________________________________________________
max_pooling2d_74 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_85 (Dropout)         (None, 8, 8, 64)        

In [65]:
ncComputor_model_1 = NCoverage(model, threshold=0.2,only_layers=[ 'dense_46']) #last hidden layer

models loaded
['dense_46']


In [66]:
# number of total inputs
print(x_train_and_adversary.shape)

(47999, 32, 32, 1)


In [67]:
import time
start_time = time.time()
nc_model_original_values_1 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[:3000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 1729.8578932285309 seconds ---


In [68]:
start_time = time.time()
nc_model_original_values_2 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[3000:6000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 1420.9735825061798 seconds ---


In [69]:
x_train_and_adversary.shape

(47999, 32, 32, 1)

In [70]:
nc_model_original_values_3 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[6000:9000]]

In [71]:
nc_values = [
    nc_model_original_values_1,
    nc_model_original_values_2,
    nc_model_original_values_3
]

In [73]:
cd C:/Users/fjdur/Documents/upc-july/data/cifar

C:\Users\fjdur\Documents\upc-july\data\cifar


In [74]:
cd C:/Users/fjdur/Documents/upc-july/data/cifar

C:\Users\fjdur\Documents\upc-july\data\cifar


In [75]:
#Save nc values
i=1
for values in nc_values:
    with open(dataset+'_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [76]:
nc_model_original_values_4 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[9000:12000]]
nc_model_original_values_5 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[12000:15000]]


In [77]:
nc_values = [
    nc_model_original_values_4,
    nc_model_original_values_5
]

In [78]:
#Save nc values
i=4
for values in nc_values:
    with open(dataset+'_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [79]:
nc_model_original_values_6 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[15000:18000]]
nc_model_original_values_7 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[18000:21000]]


In [80]:
nc_values = [
    nc_model_original_values_6,
    nc_model_original_values_7
]

In [81]:
#Save nc values
i=6
for values in nc_values:
    with open(dataset+'_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [82]:
start_time = time.time()
nc_model_original_values_8 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[21000:24000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_9 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[24000:27000]]
print("--- %s seconds ---" % (time.time() - start_time))

nc_model_original_values_10 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[27000:30000]]
print("--- %s seconds ---" % (time.time() - start_time))


--- 1219.9750046730042 seconds ---
--- 2907.314273118973 seconds ---
--- 4133.049585819244 seconds ---


In [83]:
nc_values = [
    nc_model_original_values_8,
    nc_model_original_values_9,
    nc_model_original_values_10
]

In [84]:
#Save nc values
i=8
for values in nc_values:
    with open(dataset+'_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

In [85]:
nc_model_original_values_11 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[30000:33000]]
nc_model_original_values_12 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[33000:36000]]
nc_model_original_values_13 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[36000:39000]]
nc_model_original_values_14 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[39000:42000]]
nc_model_original_values_15 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[42000:45000]]
nc_model_original_values_16 = [ncComputor_model_1.NC(tf.expand_dims(x, axis=0)) for x in x_train_and_adversary[45000:]]



In [86]:
nc_values = [
    nc_model_original_values_11,
    nc_model_original_values_12,
    nc_model_original_values_13,
    nc_model_original_values_14,
    nc_model_original_values_15,
    nc_model_original_values_16,
    
]

In [87]:
x_train_and_adversary.shape

(47999, 32, 32, 1)

In [88]:
#Save nc values
i=11
for values in nc_values:
    with open(dataset+'_nc_values_'+str(i)+'.npy', 'wb') as f:
        np.save(f, np.array(values))
    i+=1

## Uncertainty

In [89]:
from tensorflow.keras import layers
from tensorflow import keras

In [90]:
# Guardar el Modelo
model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_"+dataset+".h5"

model.save(model_dir)



In [91]:
model.evaluate(x_test,y_test)

188/188 [==============================] - 3s 17ms/step - loss: 0.5240 - accuracy: 0.8259


[0.524002730846405, 0.8258623480796814]

In [92]:
model_dir = "C:/Users/fjdur/Documents/upc-july/models/model_"+dataset+".h5"

model_original_functional = keras.models.load_model(model_dir)

In [93]:
model_original_functional.evaluate(x_test,y_test)

188/188 [==============================] - 3s 17ms/step - loss: 0.5240 - accuracy: 0.8259


[0.524002730846405, 0.8258623480796814]

In [94]:
try:
  import uncertainty_wizard as uwiz
except ModuleNotFoundError as e:
  # Uncertainty wizard was not installed. Install it now (we're probably on colab)
  !pip install uncertainty_wizard
  import uncertainty_wizard as uwiz

In [95]:
type(model_original_functional)

tensorflow.python.keras.engine.sequential.Sequential

In [96]:
stochastic_model2 = uwiz.models.stochastic_from_keras(model_original_functional)


In [97]:
quantifiers = ['deep_gini']
results_1 = stochastic_model2.predict_quantified(x_train_and_adversary,
                                   quantifier=quantifiers,
                                   batch_size=64,
                                   sample_size=32,
                                   verbose=1)

750/750 [==============================] - 21s 28ms/step


In [98]:
quantifiers = [ 'softmax_entropy']
results_2 = stochastic_model2.predict_quantified(x_train_and_adversary,
                                   quantifier=quantifiers,
                                   batch_size=64,
                                   sample_size=32,
                                   verbose=1)

750/750 [==============================] - 22s 29ms/step


In [99]:
len(x_train_and_adversary)

47999

In [100]:
#deep_gini_values = results[0][1]
#softmax_values = results[1][1]

deep_gini_values = results_1[0][1]
softmax_values = results_2[0][1]

In [78]:
print(deep_gini_values.shape)
print(softmax_values.shape)

(14224,)
(14224,)


In [101]:
save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/deep_gini_values.npy"
np.save(save_dir,np.array(deep_gini_values))

save_dir = "C:/Users/fjdur/Documents/upc-july/data/"+dataset+"/softmax_values.npy"
np.save(save_dir,np.array(softmax_values))

In [80]:
softmax_values[-15:]

array([0.95119286, 0.9268266 , 2.3521495 , 1.5793939 , 1.4716263 ,
       1.624482  , 1.2282022 , 1.4915899 , 0.32582468, 1.8323919 ,
       1.1253238 , 0.49804133, 1.5765084 , 1.9983608 , 1.4013311 ],
      dtype=float32)

In [81]:
softmax_values[:15]

array([1.9636624 , 1.1286783 , 0.75007904, 0.64505386, 1.4529158 ,
       0.96874285, 0.1260533 , 0.12664267, 0.16722937, 0.90813065,
       0.6678082 , 0.47133207, 0.03909138, 0.13503881, 0.02686379],
      dtype=float32)

In [76]:
y_test_and_adversary[:10]

array([[0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [ ]:
# results[0][0] contains the point-predictions from the 'pcs' quantifier
#     (i.e., argmax of a single non-randomized network forward pass)
# results[0][1] contains the prediction confidence scores for these predictions

# results[1][0] contains the predictions from the 'mean_softmax' quantifier
#     (i.e., argmax of 32 averages forward pass samples on the randomized DNN)
# results[1][1] contains the corresponding confidence
#     (i.e., the average softmax value of the class with the highest average softmax value)

In [30]:
print(results[0][0])

[7 9 2 ... 8 4 9]


In [82]:
print(results_2[0][1][:15])
print(results_2[0][1][-15:])

[1.9636624  1.1286783  0.75007904 0.64505386 1.4529158  0.96874285
 0.1260533  0.12664267 0.16722937 0.90813065 0.6678082  0.47133207
 0.03909138 0.13503881 0.02686379]
[0.95119286 0.9268266  2.3521495  1.5793939  1.4716263  1.624482
 1.2282022  1.4915899  0.32582468 1.8323919  1.1253238  0.49804133
 1.5765084  1.9983608  1.4013311 ]


In [83]:
print(results_1[0][1][:15])
print(results_1[0][1][-15:])

[0.6519574  0.35906863 0.22482598 0.19670624 0.47535992 0.39142632
 0.02635032 0.03382593 0.04328644 0.33758348 0.23817265 0.12240326
 0.00700969 0.03394562 0.00443   ]
[0.42461354 0.33367443 0.77483237 0.566026   0.48991388 0.58239764
 0.46859473 0.5567219  0.09249663 0.6583172  0.40377843 0.1521892
 0.518263   0.6998366  0.4969082 ]
